In [1]:
from pylab import *
from sympy import *
from sympy.solvers import solve
import numpy as np 
import transforms3d as tf3d


init_printing()

# Initialisation des fonctions lambdifier pour la simulation
## Lambdification des calculs de Cd et Cl

In [2]:
##################################################### Calcul de l'angle d'attaque et calcul du coefficient sigma qui donne la transition entre le modèle flate plate te small angle #################################
def compute_alpha_sigma(dragDirection, liftDirection, frontward_Body, VelinLDPlane, alpha_0, alpha_s, delta_s): 
    calpha= np.vdot(dragDirection, frontward_Body)
    absalpha= -np.arccos(calpha)
    signalpha = np.sign(np.vdot(liftDirection, frontward_Body)) 
    if np.linalg.norm(VelinLDPlane)>1e-7 :
        alpha = signalpha*absalpha 
    else :
        alpha=0
    if abs(alpha)>0.5*np.pi:
        if alpha>0 :alpha=alpha-np.pi 
        else: alpha=alpha+np.pi
    if alpha>=alpha_s+delta_s-alpha_0:
         sigma=0.0
    elif alpha>=alpha_s-alpha_0:
         sigma=0.5*(1.0+np.cos(np.pi*(alpha+alpha_0-alpha_s)/delta_s))
    elif alpha>=-alpha_s-alpha_0:
         sigma=1.0
    elif alpha>=-alpha_s-delta_s-alpha_0:
         sigma=0.5*(1.0+np.cos(np.pi*(alpha+alpha_0+alpha_s)/delta_s))
    else:
         sigma=0.0
    return alpha, sigma


# Initialisation des paramètres pour les équations symbolique
CL_1_sa = Symbol('C^{sa}_{L,1}',real=True)             # Coeff rechercher

CLsa = Symbol('C_L^{sa}',real=True)
CLfp = Symbol('C_L^{fp}',real=True)
C_L = Symbol('C_L',real=True)

CD_0_sa = Symbol('C^{sa}_{D,0}',real=True)             # Coeff rechercher
CD_1_sa = Symbol('C^{sa}_{D,1}',real=True)             # Coeff rechercher
CD_0_fp = Symbol('C^{fp}_{D,0}',real=True)             # Coeff rechercher
CD_1_fp = Symbol('C^{fp}_{D,1}',real=True)             # Coeff rechercher

CDsa = Symbol('C_D^{sa}',real=True)
CLfp = Symbol('C_D^{fp}',real=True)
C_D = Symbol('C_D',real=True)                       

k_0 = Symbol('k_0', real=True)                         # coeff rechercher
k_1 = Symbol('k_1', real=True)                         # coeff rechercher
k_2 = Symbol('k_2', real=True)                         # coeff rechercher

sigma = Symbol('sigma',real=True)                     # Coef de transistion small angle / flat plate

delta_0 = Symbol('delta_0',real=True)                 # Angle des ailerons
delta_s = Symbol('delta_s', real=True)                # Coeff rechercher : largeur du stall

alpha_0 = Symbol('alpha_0',real=True)                 # Coeff rechercher 
alpha_s = Symbol('alpha_s',real=True)                 # Coeff rechercher
alpha = Symbol('alpha',real=True)                     # angle d'attaque

theta={'alpha_stall': alpha_s,
              'largeur_stall':delta_s,
              'cd1sa': CD_1_sa,
              'cl1sa':CL_1_sa,
              'cd0sa':CD_0_sa,
              'cd1fp':CD_1_fp,
              'cd0fp':CD_0_fp,
              'coeff_drag_shift':k_0,
              'coeff_lift_shift':k_1,
              'coef_lift_gain':k_2}

##################################################### Calcul des fonctions ####################################################
CL_sa = 1/2 * CL_1_sa * sin(2*(alpha + alpha_0))
CD_sa = CD_0_sa + CD_1_sa * sin(alpha + alpha_0)*sin(alpha + alpha_0)

CL_fp = 1/2 * CD_1_fp * sin(2*(alpha + alpha_0))
CD_fp = CD_0_fp + CD_1_fp * sin(alpha + alpha_0)*sin(alpha + alpha_0)

C_L = CL_fp + sigma*(CL_sa - CL_fp) + k_2 * sin(delta_0)
C_D = CD_fp + sigma*(CD_sa - CD_fp)

Coeff = [C_L, C_D]    

grad_C_L = Matrix([C_L]).jacobian([i for i in theta.values()])
grad_C_D = Matrix([C_D]).jacobian([i for i in theta.values()])

#################################################### Lambdification des fonctions que nous allons utiliser en simulation. ####################################################
CL_function = lambdify((alpha, sigma, alpha_0, delta_0, CL_1_sa, CD_1_fp, k_2), C_L)          
CD_function = lambdify((alpha, sigma, alpha_0, CD_0_fp, CD_0_sa, CD_1_sa, CD_1_fp, k_0), C_D)

##################################################### Lamdification de la fonction comprenant les deux coefficients aérodynamique, retourne sous la forme [Cl, Cd] ###################################
Coeff_function = lambdify((alpha, sigma, alpha_0, delta_0, CL_1_sa, CD_1_fp, k_2, CD_0_fp, CD_0_sa, CD_1_sa, CD_1_fp), Coeff) 

grad_Cl_function = lambdify((alpha, sigma, alpha_0, delta_0, CL_1_sa, CD_1_fp, k_2, CD_0_fp, CD_0_sa, CD_1_sa, CD_1_fp), grad_C_L)
grad_Cd_function = lambdify((alpha, sigma, alpha_0, delta_0, CL_1_sa, CD_1_fp, k_2, CD_0_fp, CD_0_sa, CD_1_sa, CD_1_fp), grad_C_D)

In [34]:
grad_C_L.T

⎡                     0                      ⎤
⎢                                            ⎥
⎢                     0                      ⎥
⎢                                            ⎥
⎢                     0                      ⎥
⎢                                            ⎥
⎢           0.5⋅σ⋅sin(2⋅α + 2⋅α₀)            ⎥
⎢                                            ⎥
⎢                     0                      ⎥
⎢                                            ⎥
⎢-0.5⋅σ⋅sin(2⋅α + 2⋅α₀) + 0.5⋅sin(2⋅α + 2⋅α₀)⎥
⎢                                            ⎥
⎢                     0                      ⎥
⎢                                            ⎥
⎢                     0                      ⎥
⎢                                            ⎥
⎢                     0                      ⎥
⎢                                            ⎥
⎣                  sin(δ₀)                   ⎦

## Lambdification des caluls des efforts aérodynamiques des ailes et des moteurs : 

In [4]:
################################################### Paramètres pour les équations symboliques pour les efforts aérodynamiques des ailes et des moteurs #######################

B_B       = Matrix([[1,0,0], [0,1,0], [0,0,1]])                              # Base dans le repère body

omega1, omega2, omega3 = symbols('\omega_1, \omega_2, \omega_3', real=True)
Omega     = Matrix([omega1, omega2, omega3])                                 # Vecteur de rotation
 
m = Symbol('m', real=True)    
alpha = Symbol('alpha',real=True)                     # angle d'attaque

r00, r01, r02, r10, r11, r12, r20, r21, r22 = symbols('r_{00}, r_{01}, r_{02}, r_{10}, r_{11}, r_{12}, r_{20}, r_{21}, r_{22}', real=True)
R         = Matrix([[r00,r01,r02], [r10,r11, r12], [r20, r21, r22]])          # Matrice de rotation

Vb1,Vb2,Vb3=symbols('V_{b1} V_{b2} V_{b3}',real=True)
v_B        = Matrix([Vb1, Vb2, Vb3])                                          # Vitesse du corps (repère drone)

Vw1,Vw2,Vw3=symbols('V_{w1} V_{w2} V_{w3}',real=True)                         # Vitesse du vent dans le repère NED 
v_W        = Matrix([Vw1, Vw2, Vw3])

xcp, ycp, zcp = symbols('x_{cp}, y_{cp}, z_{cp}')
X_cp     = Matrix([xcp, ycp, zcp])                                            # Position du centre de poussé d'un corps dans le repère body
alpha_0 = Symbol('alpha_0',real=True)                   # Coeff rechercher                                        

alpha_s = Symbol('alpha_s',real=True)                 # Coeff rechercher
sigma = Symbol('sigma',real=True)                     # Coef de transistion small angle / flat plate

delta_0 = Symbol('delta_0',real=True)                 # Angle des ailerons
delta_s = Symbol('delta_s', real=True)                # Coeff rechercher : largeur du stall

A         = Symbol('A', real=True)                                            # 0.5 * rho * S 

VxNED, VyNED , VzNED = symbols('V_{NED_x} V_{NED_y} V_{NED_z}') 
Air_speed_earth  = Matrix([VxNED, VyNED , VzNED])                             # air speed dans le repère NED

VxBody, VyBody, VzBody = symbols('V_{body_x} V_{body_y} V_{body_z}') 
Air_speed_body   = Matrix([VxBody, VyBody, VzBody])                           # air speed dans le repère body

omega,ct=symbols('\omega,C_t',real=True)                                      # vitesse de rotation des moteurs, et coefficient de poussé des helices
Cd, Cl, Ct, Cq=symbols('C_d,C_l,C_t,C_q',real=True)                           # vitesse de rotation des moteurs, et coefficient de poussé des helices

cl1, cl2, cl3, cl4, cl5 = symbols('cl_1, cl_2, cl_3, cl_4, cl_5', real=True)
cd1, cd2, cd3, cd4, cd5 = symbols('cd_1, cd_2, cd_3, cd_4, cd_5', real=True)

motor_axis_in_body_frame = Matrix([1,0,0])                                    # Axe des moteurs, ici placé en mode avion

ch = Symbol('c_h')

omega_rotor = symbols('\omega_{rotor}', real=True)                            # Vitesse de rotation des moteurs
spinning_sense = symbols('\epsilon_{spin}', int=True)                         # Sens de rotation d'un moteur [-1 ou 1]

frontward_B = B_B[:,0]
upward_B    = B_B[:,2]
crossward_B = B_B[:,1]

c45, s45 = np.cos(45*np.pi/180), np.sin(45*np.pi/180)

########################################### Les différentes fonctions utilisé pour générer les différentes équations symboliques ###################################################

def GenDirectForceWing(Omega, cp, vB, vW, R, crossward_Body):
    # Cette fonction permet d'obtenir les directions des efforts de portances et de trainé en fonction des vitesses, et de l'orientation dans le repère NED.
    Air_speed_earth = vB - vW 
    Air_speed_body  = (R.T* Air_speed_earth) - cp.cross(Omega)
    VelinLDPlane    = Air_speed_body - Air_speed_body.dot(crossward_Body.T) * crossward_Body
    
    dragDirection = -VelinLDPlane / VelinLDPlane.norm()  #if VelinLDPlane_norm > VelLim else Matrix([0,0,0])
    liftDirection = -crossward_Body.cross(dragDirection) #if crossward_NED.norm() > VelLim else Matrix([0,0,0])

    return VelinLDPlane, dragDirection, liftDirection

def GenForceWing(A, VelinLDPlane, dragDirection, liftDirection, Cd, Cl, cp):
    # Cette fonction permet de générer les forces aerodynamique d'une aile dans son repère.
    D = A * VelinLDPlane.norm()**2 * dragDirection * Cd
    L = A * VelinLDPlane.norm()**2 * liftDirection * Cl

    F_wing = L+D 
    Torque_wing =  cp.cross(F_wing)

    return F_wing, Torque_wing

def Generate_grad_Force_wing(A, VelinLDPlane, dragDirection, liftDirection, Cd_grad, Cl_grad, cp):
    
    D = A * VelinLDPlane.norm()**2 * dragDirection * Cd_grad.T
    L = A * VelinLDPlane.norm()**2 * liftDirection * Cl_grad.T

    grad_Force_Wing = L+D 
    grad_Torque_Wing=Matrix([])
    for col in range(max(grad_Force_Wing.shape)):
        grad_Torque_Wing = grad_Torque_Wing.col_insert(col, cp.cross(grad_Force_Wing.col(col)))    
    
    return grad_Force_Wing, grad_Torque_Wing

def Generate_Sum_Force_wing(A_list, Omega, cp_list, R_list, vB, vW,  Cd_list, Cl_list, crossward_body, r_queue, r_queue_neg):
    # Cette function permet de généer l'équation complète de la somme des forces pour les différentes surfaces portantes 
    p = 0
    Sum_Force_Wing = Matrix([0,0,0])
    Sum_Torque_Wing =  Matrix([0,0,0])
    for i in cp_list:
        VelinLDPlane, dragDirection, liftDirection= GenDirectForceWing(Omega, i, vB, vW, R_list[p], crossward_body)
        if p == 2 :
            # Comme la fonction GenForceWing donne les efforts des ailes dans leur repère propre, on doit passer par les matrice de rotation pour les ailes de la queue
            F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd_list[p], Cl_list[p], Matrix([0,0,0]))
            Sum_Force_Wing  = Sum_Force_Wing +  r_queue.T  @ F_wing
            Sum_Torque_Wing = Sum_Torque_Wing + i.cross(r_queue.T  @ F_wing)
        elif p == 3 :
            F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd_list[p], Cl_list[p], Matrix([0,0,0]))
            Sum_Force_Wing  = Sum_Force_Wing +  r_queue_neg.T  @ F_wing
            Sum_Torque_Wing = Sum_Torque_Wing + i.cross(r_queue_neg.T  @ F_wing)
        else:
            F_wing, Torque_wing =  GenForceWing(A_list[p], VelinLDPlane, dragDirection, liftDirection, Cd_list[p], Cl_list[p], i)
            Sum_Force_Wing  = Sum_Force_Wing +  F_wing
            Sum_Torque_Wing = Sum_Torque_Wing + Torque_wing
        p+=1

    return Sum_Force_Wing, Sum_Torque_Wing

def GenForceMoteur(Omega, ct, cq, omega_rotor, cp, vB, vW, ch, R, motor_axis_in_body_frame, spinning_sense):
    ## Cette fonction calcule les effort produit par un rotor sur le drone en fonction de son sens de rotation et de sa localisation, les efforts sont donnés
    ## dans le repère inertiel. l'axe des moteur est placé suivant l'axe x du drone (mode avion seulement)
    Air_speed_earth = vB - vW
    air_speed_in_rotor_frame = (R.T* Air_speed_earth) - cp.cross(Omega)
    Axial_speed = air_speed_in_rotor_frame.dot(motor_axis_in_body_frame)
    lat_speed = air_speed_in_rotor_frame - (Axial_speed * (motor_axis_in_body_frame))
        
    T = ct*omega_rotor**2
    H = ch * omega_rotor
    
    T_vec = T * motor_axis_in_body_frame - H * lat_speed
    
    torque = - omega_rotor * cq * lat_speed
    torque = - spinning_sense * cq * T * motor_axis_in_body_frame 
    torque_at_body_center = torque + cp.cross(T_vec.T)
        
    return T_vec, torque_at_body_center

def Generate_Sum_Force_Moteur(Omega, ct, cq, omega_rotor, cp_list, vB, vW, ch, R, motor_axis_in_body_frame_list, spinning_sense_list):
    # Calcul des forces des moteurs sur le drone, génère toutes les forces, ainsi que le couple appliqué au centre de gravité du drone, dans le repère inertiel
    p = 0
    Sum_Force_Rotor = Matrix([0,0,0])
    Sum_Torque_Rotor =  Matrix([0,0,0])
    for cp in cp_list:
        F_rotor, Q_rotor = GenForceMoteur(Omega, ct, cq, omega_rotor, cp, vB, vW, ch, R, motor_axis_in_body_frame_list, spinning_sense_list[p])        
        Sum_Force_Rotor  = Sum_Force_Rotor + F_rotor
        Sum_Torque_Rotor = Sum_Torque_Rotor + Q_rotor
        p+=1

    return Sum_Force_Rotor, Sum_Torque_Rotor

############################################## Initilisation des paramètres pour la génération des équations des ailes ###########################
c45, s45 = np.cos(45*np.pi/180), np.sin(45*np.pi/180)
r = np.array(((1,0, 0),(0,c45,-s45),(0,s45, c45)))
r_neg     = np.array(((1,0, 0), (0,c45, s45),(0,-s45, c45)))
R_list = [R, R,  R * r,  R *r_neg, R]      # Liste des matrices de rotation de chaque surface portante du drone, seul les éléments de la queue (element 3 et 4) ne sont pas dans le repère inertiel. 

cp1x,cp1y, cp1z, cp2x,cp2y,cp2z, cp3x,cp3y,cp3z, cp4x,cp4y,cp4z,cp5x,cp5y,cp5z = symbols('cp1x,cp1y, cp1z, cp2x,cp2y,cp2z, cp3x,cp3y,cp3z, cp4x,cp4y,cp4z,cp5x,cp5y,cp5z', real=True)
cp_list = [Matrix([cp1x,cp1y, cp1z]), Matrix([cp2x,cp2y,cp2z]), Matrix([cp3x,cp3y,cp3z]), Matrix([cp4x,cp4y,cp4z]), Matrix([cp5x,cp5y,cp5z])]

A_list = [0.03* 1.292 * 0.5, 0.03* 1.292 * 0.5, 0.01* 1.292 * 0.5, 0.01* 1.292 * 0.5, 0.04* 1.292 * 0.5]  # Liste des 1/2 * rho * S pour chaque surface
Cd_list_init = [cd1, cd2, cd3, cd4, cd5]
Cl_list_init = [cl1, cl2, cl3, cl4, cl5]

for i in range(10):
  globals()["cdgrad_%i"%i]=Symbol('cd_grad_%i'%i, real=True)
  globals()["clgrad_%i"%i]=Symbol('cd_grad_%i'%i, real=True)

Cd_grad = Matrix(["cdgrad_%i"%i for i in range(10)])
Cl_grad = Matrix(["clgrad_%i"%i for i in range(10)])
# Initialisation pour les équations des rotors
cp_list_rotor = [Matrix([0.2,0.45,0]), Matrix([0.2,-0.45,0])]
spinning_sense_list = [1,-1]


############################################### Génération des équations : ######################################

####################### Création des équations des efforts pour une aile et un seul rotor 
 # Ces équation servent uniquement pour le calcul de l'angle d'attaque pour le calcul des coeffficient aérodynamique Cd et Cl, il n'interviennent pas directement dans le calcul des efforts. 
VelinLDPlane, dragDirection, liftDirection=  GenDirectForceWing(Omega, X_cp, v_B, v_W, R, crossward_B)
Force_wing, Torque_wing = GenForceWing(A, VelinLDPlane, dragDirection, liftDirection, Cd, Cl, X_cp)
                                       
Force_rotor, Torque_rotor = GenForceMoteur(Omega, Ct, Cq, omega_rotor, X_cp, v_B, v_W,ch, R, motor_axis_in_body_frame, spinning_sense)

####################### Génération des équations pour le gradient des forces pour une ailes. 
Grad_Force_wing, Grad_Torque_wing = Generate_grad_Force_wing(A, VelinLDPlane, dragDirection, liftDirection, Cd_grad, Cl_grad, X_cp)

Grad_Force_wing_function = lambdify((A, Cd_grad, Cl_grad, v_B, v_W, Omega, X_cp, R), Grad_Force_wing, 'numpy')
Grad_Torque_wing_function = lambdify((A, Cd_grad, Cl_grad, v_B, v_W, Omega, X_cp, R), Grad_Torque_wing, 'numpy')

## Lambdification des fonction
Force_wing_function = lambdify((A, Cd, Cl, v_B, v_W, Omega, X_cp, R), Force_wing, 'numpy')
Torque_wing_function = lambdify((A, Cd, Cl, v_B, v_W, Omega, X_cp, R), Torque_wing, 'numpy')

VelinLDPlane_function = lambdify((Omega, X_cp, v_B, v_W, R), VelinLDPlane, 'numpy')
dragDirection_function = lambdify((Omega, X_cp, v_B, v_W, R), dragDirection, 'numpy')
liftDirection_function = lambdify((Omega, X_cp, v_B, v_W, R), liftDirection, 'numpy')

Force_rotor_function = lambdify((Ct, omega_rotor, v_B, v_W, R, Omega, X_cp), Force_rotor)
Torque_rotor_function = lambdify((Cq, Ct, omega_rotor, Omega, R, v_B, v_W, X_cp, spinning_sense), Torque_rotor)

##################### Sommes des efforts des surfaces portantes
  # Ces équations sont celles qui vont être utilisé en simulation, elle génèrent les efforts des différents moteurs et surfaces portantes sur le drone dans le repère intertiel. 
Sum_F_wing, Sum_T_wing = Generate_Sum_Force_wing(A_list, Omega, cp_list, R_list, v_B, v_W, Cd_list_init, Cl_list_init, crossward_B, r, r_neg)
#Sum_F_wing.simplify()    # On ne fait pas les simplifications car elles prennent trop de temps et ne permettent pas de gagner en rapidité de manière conséquente.
#Sum_T_wing.simplify()

## Lambdification des fonctions  
Sum_F_wing_function = lambdify((Omega, R, v_B, v_W, cp_list, Cd_list_init, Cl_list_init), Sum_F_wing, 'numpy')
Sum_T_wing_function = lambdify((Omega, R, v_B, v_W, cp_list, Cd_list_init, Cl_list_init), Sum_T_wing, 'numpy')

##################### Sommes des efforts des moteurs
  # Ces équations donnes les équations de la sommes des efforts des moteurs sur le drone dans le repère intertiel
Sum_F_rotor, Sum_T_rotor = Generate_Sum_Force_Moteur(Omega, Ct, Cq, omega_rotor, cp_list_rotor, v_B, v_W, ch, R, motor_axis_in_body_frame, spinning_sense_list)
Sum_F_rotor.simplify()
Sum_T_rotor.simplify()


grad_Sum_F_rotor = Matrix([Sum_F_rotor]).jacobian([i for i in theta.values()])
grad_Sum_T_rotor = Matrix([Sum_T_rotor]).jacobian([i for i in theta.values()])

grad_Sum_rotor = [grad_Sum_F_rotor, grad_Sum_T_rotor]

grad_Sum_rotor = lambdify((Omega, R, v_B, v_W, Ct, Cq, ch, omega_rotor), grad_Sum_rotor, 'numpy')

## Lamdification des fonctions
Sum_F_rotor_function = lambdify((Omega, R, v_B, v_W, Ct, Cq, ch, omega_rotor), Sum_F_rotor, 'numpy')
Sum_T_rotor_function = lambdify((Omega, R, v_B, v_W, Ct, Cq, ch, omega_rotor), Sum_T_rotor, 'numpy')

##################### Equation des forces et des couples complètes  #################################################    
        # Il s'agit de l'équation principale utilisé en simulation, elle regroupe et somme simplement les equations ci-dessus.
Effort_Aero = [Sum_F_rotor + Sum_F_wing, Sum_T_rotor + Sum_T_wing]

# Somme des forces et des couples des ailes et des moteurs  ur le drone dans le repère intertiel:
Effort_Aero_function = lambdify((Omega, R, v_B, v_W, cp_list, Cd_list_init, Cl_list_init, Ct, Cq, ch, omega_rotor), Effort_Aero, 'numpy')

# Fonction gradient : 

In [5]:
for i in range(10):
  globals()["epsilon_%s"%i]=Symbol('epsilon_%s'%i, real=True)
Epsilon = Matrix(["epsilon_%s"%i for i in range(10)])

params_effort = [Omega, R, v_B, v_W, cp_list, Cd_list_init, Cl_list_init, Ct, Cq, ch, omega_rotor]

Theta={"alpha_stall" : 0.3391428111 ,                     \
       "largeur_stall" : 30.0*np.pi/180,                  \
       "cd0sa" : 0.045,\
       "cd0fp" : 0.045,\
       "cd1sa" : 4.55, \
       "cl1sa" : 5, \
       "cd1fp" : 2.5, \
       "coeff_drag_shift": 0.5, \
       "coeff_lift_shift": 0.5, \
       "coef_lift_gain": 0.5}


epsilon={'alpha_stall': epsilon_0,
              'largeur_stall':epsilon_1,
              'cd1sa': epsilon_2,
              'cl1sa':epsilon_3,
              'cd0sa':epsilon_4,
              'cd1fp':epsilon_5,
              'cd0fp':epsilon_6,
              'coeff_drag_shift':epsilon_7,
              'coeff_lift_shift':epsilon_8,
              'coef_lift_gain':epsilon_9}

alpha_0_2, alpha_0_3, alpha_0_4, alpha_0_5 = symbols('alpha_0_2, alpha_0_3, alpha_0_4, alpha_0_5', real=True)
alpha_0_list    =  np.array([0.06,0.06,0,0,0.06])

masse = 2
R = np.eye(3)
r = np.array(((1,0, 0),(0,c45,-s45),(0,s45, c45)))
r_neg = np.array(((1,0, 0), (0,c45, s45),(0,-s45, c45)))

R_list = [R, R, R@r, R@r_neg, R]

cp_list = [np.array([0,0.45,0], dtype=np.float), np.array([0,-0.45,0], dtype=np.float), \
           np.array([-0.5,0.15,0], dtype=np.float), np.array([-0.5,-0.15,0], dtype=np.float),np.array([0,0,0], dtype=np.float)]

v_W = np.array([0,0,0.0], dtype=np.float)
g = np.array([0,0,9.81])
v_B = np.array([10,0,0])
Cl_list= [0,0,0,0,0]
Cd_list= [0,0,0,0,0]
A_list = [0.03* 1.292 * 0.5, 0.03* 1.292 * 0.5, 0.01* 1.292 * 0.5, 0.01* 1.292 * 0.5, 0.04* 1.292 * 0.5]  # Liste des 1/2 * rho * S pour chaque surface


In [33]:
def gradien_F(theta, A, Omega, cp_list, v_B, v_W, R_list, alpha_0_list):
    Grad_Sum_F =[0 for i in range(len(theta))]
    Grad_Sum_T =[0 for i in range(len(theta))]

    for p,cp in enumerate(cp_list) : 
        ###### Test du calcul de l'angle d'attaque et des efforts de chque surface portante sue le drone #######
        VelinLDPlane = VelinLDPlane_function(Omega, cp, v_B, v_W, R_list[p].flatten())
        dragDirection = dragDirection_function(Omega, cp, v_B, v_W, R_list[p].flatten())
        liftDirection = liftDirection_function(Omega, cp, v_B, v_W, R_list[p].flatten())

        alpha, sigma = compute_alpha_sigma(dragDirection, liftDirection, np.array([1,0,0]), VelinLDPlane, alpha_0_list[p], theta['alpha_stall'], theta['largeur_stall'])
        
        if p ==3 :
            delta0 = -15*np.pi/180 *0
            Cl_list[p] = grad_Cl_function(alpha + (theta["coeff_lift_shift"]*delta0), sigma, alpha_0_list[p], delta0, theta['cl1sa'], theta['cd1fp'], theta['coef_lift_gain'], theta['cd0fp'], theta['cd0sa'], theta['cd1sa'], theta['cd1fp'])
            Cd_list[p] = grad_Cd_function(alpha + (theta["coeff_drag_shift"]*delta0), sigma, alpha_0_list[p], delta0, theta['cl1sa'], theta['cd1fp'], theta['coef_lift_gain'], theta['cd0fp'], theta['cd0sa'], theta['cd1sa'], theta['cd1fp'])
 
        elif p==2:
            delta0 = -15*np.pi/180*0
            Cl_list[p] = grad_Cl_function(alpha + (theta["coeff_lift_shift"]*delta0), sigma, alpha_0_list[p], delta0, theta['cl1sa'], theta['cd1fp'], theta['coef_lift_gain'], theta['cd0fp'], theta['cd0sa'], theta['cd1sa'], theta['cd1fp'])
            Cd_list[p] = grad_Cd_function(alpha + (theta["coeff_drag_shift"]*delta0), sigma, alpha_0_list[p], delta0, theta['cl1sa'], theta['cd1fp'], theta['coef_lift_gain'], theta['cd0fp'], theta['cd0sa'], theta['cd1sa'], theta['cd1fp'])
           
        else:
            delta0=0
            Cl_list[p] = grad_Cl_function(alpha + (theta["coeff_lift_shift"]*delta0), sigma, alpha_0_list[p], delta0, theta['cl1sa'], theta['cd1fp'], theta['coef_lift_gain'], theta['cd0fp'], theta['cd0sa'], theta['cd1sa'], theta['cd1fp'])
            Cd_list[p] = grad_Cd_function(alpha + (theta["coeff_drag_shift"]*delta0), sigma, alpha_0_list[p], delta0, theta['cl1sa'], theta['cd1fp'], theta['coef_lift_gain'], theta['cd0fp'], theta['cd0sa'], theta['cd1sa'], theta['cd1fp'])

        grad_Force_wing  = Grad_Force_wing_function(A[p], Cd_list[p].flatten(), Cl_list[p].flatten(), v_B, v_W, Omega, cp, R_list[p].flatten())
        grad_Torque_wing = Grad_Torque_wing_function(A[p], Cd_list[p].flatten(), Cl_list[p].flatten(), v_B, v_W, Omega, cp, R_list[p].flatten())
        
        for col in range(len(theta)):
            Grad_Sum_F[col] = R_list[p]@ np.transpose(grad_Force_wing[:,col]) + masse * g
            Grad_Sum_T[col] = R_list[p]@np.transpose(grad_Torque_wing[:,col])
    grad_Sum_rotor_force, grad_Sum_rotor_torque = grad_Sum_rotor(Omega,  R_list[p].flatten(), v_B, v_W,  0.5e-4, 1e-8, 1e-4, 230)

    return np.reshape(Grad_Sum_F,(3,10)) +grad_Sum_rotor_force, np.reshape(Grad_Sum_T,(3,10)) +grad_Sum_rotor_torque

F_test, T_test =gradien_F(Theta, A_list, np.array([0,0,0]), cp_list, v_B, v_W, R_list, alpha_0_list)


# Test des fonctions

In [7]:
R = np.eye(3)
r = np.array(((1,0, 0),(0,c45,-s45),(0,s45, c45)))
r_neg = np.array(((1,0, 0), (0,c45, s45),(0,-s45, c45)))

R_list = [R, R, R@r, R@r_neg, R]

cp_list = [np.array([0,0.45,0], dtype=np.float), np.array([0,-0.45,0], dtype=np.float), \
           np.array([-0.5,0.15,0], dtype=np.float), np.array([-0.5,-0.15,0], dtype=np.float),np.array([0,0,0], dtype=np.float)]

A = A_list
##################################################### Initialisation des coefs aéro : 
k1 = 10
k2 = 10
k0 = 10
cd1fp = 2.5
cl1fp = cd1fp

cl1sa = 5
cd1sa = 4.55


cd0sa = 0.045
cd0fp = cd0sa

alpha0 = 0.06
alpha_0_list = np.array([alpha0,alpha0,0.0,0.0,alpha0])

alpha_s = 0.3391428111
delta_s = 30.0*np.pi/180 
v_W = np.array([0,0,0.0], dtype=np.float)
g = np.array([0,0,9.81])
V_essai = np.array([20,0,0])
Cl_list= [0,0,0,0,0]
Cd_list= [0,0,0,0,0]
delta0=0
p=0
O =np.array([0,0.5,0])*0
Force_wing_essai = np.array([0,0,0])
Torque_wing_essai = np.array([0,0,0])

#################################################### Test des fonctions dans le cas d'une seul itérations, sans compter la dynamique (masse, inertie) du drone ####################################################

for p,cp in enumerate(cp_list) : 
        ###### Test du calcul de l'angle d'attaque et des efforts de chque surface portante sue le drone #######
        VelinLDPlane = VelinLDPlane_function(O,cp, V_essai, np.array([0,0,0]), R_list[p].flatten())
        dragDirection = dragDirection_function(O,cp, V_essai, np.array([0,0,0]),R_list[p].flatten())
        liftDirection = liftDirection_function(O,cp, V_essai, np.array([0,0,0]), R_list[p].flatten())
        
        alpha, sigma = compute_alpha_sigma(dragDirection, liftDirection, np.array([1,0,0]), VelinLDPlane, alpha_0_list[p], alpha_s, delta_s)
        
        if p ==3 :
            delta0 = -15*np.pi/180 *0
            Cl_list[p] = Coeff_function(alpha + (k1*delta0), sigma, alpha_0_list[p], delta0, cl1sa, cd1fp, k2, cd0fp, cd0sa, cd1sa, cd1fp)[0]
            Cd_list[p] = Coeff_function(alpha + (k0*delta0), sigma, alpha_0_list[p], delta0, cl1sa, cd1fp, k2, cd0fp, cd0sa, cd1sa, cd1fp)[1]
            F  =  (Force_wing_function(A[p], Cd_list[p],  Cl_list[p], V_essai,  np.array([0,0,0]), O,  np.array([0,0,0]), R_list[p].flatten()).flatten())
            Torque_wing_essai  = Torque_wing_essai + np.cross(cp,F)
            Force_wing_essai = Force_wing_essai + F
        elif p==2:
            delta0 = -15*np.pi/180*0
            Cl_list[p] = Coeff_function(alpha + (k1*delta0), sigma, alpha_0_list[p], delta0, cl1sa, cd1fp, k2, cd0fp, cd0sa, cd1sa, cd1fp)[0]
            Cd_list[p] = Coeff_function(alpha + (k0*delta0), sigma, alpha_0_list[p], delta0, cl1sa, cd1fp, k2, cd0fp, cd0sa, cd1sa, cd1fp)[1]
            F =  (Force_wing_function(A[p], Cd_list[p],  Cl_list[p], V_essai,  np.array([0,0,0]), O, np.array([0,0,0]), R_list[p].flatten()).flatten())
            Torque_wing_essai  = Torque_wing_essai + np.cross(cp,F)
            Force_wing_essai =Force_wing_essai + F

        else:
            Cl_list[p] = Coeff_function(alpha + (k1*delta0), sigma, alpha_0_list[p], delta0, cl1sa, cd1fp, k2, cd0fp, cd0sa, cd1sa, cd1fp)[0]
            Cd_list[p] = Coeff_function(alpha + (k0*delta0), sigma, alpha_0_list[p], delta0, cl1sa, cd1fp, k2, cd0fp, cd0sa, cd1sa, cd1fp)[1]
            Force_wing_essai = Force_wing_essai+  Force_wing_function(A[p], Cd_list[p],  Cl_list[p], V_essai,  np.array([0,0,0]), O, cp, R_list[p].flatten()).flatten()
            Torque_wing_essai = Torque_wing_essai+  Torque_wing_function(A[p], Cd_list[p],  Cl_list[p], V_essai,  np.array([0,0,0]), O, cp, R_list[p].flatten()).flatten()
            
##### Test des fonctions qui calcul la somme des forces des moteurs et des ailes ########
torque_somme1 = Sum_T_wing_function(O, np.eye(3).flatten(), V_essai, np.array([0,0,0]), cp_list, Cd_list, Cl_list)     
force_somme1 = Sum_F_wing_function(O, np.eye(3).flatten(), V_essai, np.array([0,0,0]), cp_list, Cd_list, Cl_list)

Sum_F_rotor1 = Sum_F_rotor_function(O, np.eye(3).flatten(), V_essai, np.array([0,0,0]), 0.5e-4, 1e-8, 1e-4, 230)
Sum_Q_rotor1 = Sum_T_rotor_function(O, np.eye(3).flatten(), V_essai, np.array([0,0,0]), 0.5e-4, 1e-8, 1e-4, 200)

##### Test de la fonction qui regroupe tout le calcul précédent : Elle calcul les efforts des moteurs et des ailes #####
Effort, Effort_T = Effort_Aero_function(O, np.eye(3).flatten(), V_essai, np.array([0,0,0]), cp_list, Cd_list, Cl_list, 2.5e-5, 1e-8, 1e-4, 200)
R @ np.transpose(Effort.flatten()) + g
Effort_T

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [-1.04083409e-17]])

# Exportation des fonctions dans un fichier nomé : 

In [9]:
import dill as dill
dill.settings['recurse'] = True

Functions = [VelinLDPlane_function, dragDirection_function, liftDirection_function, compute_alpha_sigma, Coeff_function, Effort_Aero_function, grad_Cl_function, grad_Cd_function, Grad_Force_wing_function, Grad_Torque_wing_function, grad_Sum_rotor]
dill.dump(Functions, open("../Simulation/function_moteur_physique", "wb"))

